In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import math

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from dataclasses import dataclass
from typing import Union, Callable, Optional
from copy import deepcopy
from scipy.optimize import root_scalar, brentq
from dataclasses import dataclass
from scipy.special import iv
from scipy.stats import bernoulli
from scipy.interpolate import RectBivariateSpline

from vol.vol import Heston

import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")


@dataclass
class StockOption:
    strike_price:    Union[float, np.ndarray]
    expiration_time: Union[float, np.ndarray]  # in years
    is_call:         bool

@dataclass
class CallStockOption(StockOption):
    def __init__(self, strike_price, expiration_time):
        super().__init__(strike_price, expiration_time, True)

@dataclass
class PutStockOption(StockOption):
    def __init__(self, strike_price, expiration_time):
        super().__init__(strike_price, expiration_time, False)

@dataclass
class HestonParameters:
    kappa:  Union[float, np.ndarray]
    gamma:  Union[float, np.ndarray]
    rho:    Union[float, np.ndarray]
    vbar:   Union[float, np.ndarray]
    v0:     Union[float, np.ndarray]
        
@dataclass
class MarketState:
    stock_price:   Union[float, np.ndarray]
    interest_rate: Union[float, np.ndarray]

# Control Interface

In [22]:
def get_len_conf_interval(data:             np.ndarray, 
                          confidence_level: float = 0.05):
    """Get the confidence interval length for a given confidence level.
    Args:
        data:             The data to compute the confidence interval for.
        confidence_level: The confidence level to use.
    
    Returns:
        The confidence interval.
    """
    return -2*sps.norm.ppf(confidence_level*0.5) * np.sqrt(np.var(data) / len(data))

def mc_price(payoff:                 Callable,
             simulate:               Callable,
             market_state:           MarketState,
             params:                 HestonParameters,
             T:                      float = 1.,
             dt:                     float = 1e-2,
             absolute_error:         float = 0.01,
             confidence_level:       float = 0.05,
             batch_size:             int   = 10_000,
             MAX_ITER:               int   = 100_000,
             control_variate_payoff: Callable = None,
             **kwargs):
    """A function that performs a Monte-Carlo based pricing of a 
       derivative with a given payoff (possibly path-dependent)
       under the Heston model.

    Args:
        payoff (Callable):                  payoff function
        simulate (Callable):                simulation engine
        market_state (MarketState):         market state
        params (HestonParameters):          Heston parameters
        T (float, optional):                Contract expiration time. Defaults to 1.. 
        absolute_error (float, optional):   absolute error of the price. Defaults to 0.01 (corresponds to 1 cent). 
        confidence_level (float, optional): confidence level for the price. Defaults to 0.05.
        batch_size (int, optional):         path-batch size. Defaults to 10_000.
        undersample_size (int, optional):   initial guess for the MC simulation error. To be deprecated. Defaults to 1000.
        time_batch_size (int, optional):    pathwise time-batch size. Defaults to 10_000.
    """

    arg = {'state':           market_state, #renamed to market_state from state
          'heston_params':   params, 
          'time':            T , 
          'dt':              dt, 
          'n_simulations':   batch_size}

    args       = {**arg, **kwargs}
    iter_count = 1   


    derivative_price_array = np.array(payoff(simulate(**args)['price']), dtype=np.float64)  

    while get_len_conf_interval(derivative_price_array) > absolute_error and iter_count < MAX_ITER:
        derivative_price_array = np.append(derivative_price_array, payoff(simulate(**args)['price']))
        iter_count+=1
        print(f"Current price: {np.mean(derivative_price_array):.4f} +/- {get_len_conf_interval(derivative_price_array):.4f}")

    print(f"Number of iterations:   {iter_count}")

    return np.mean(derivative_price_array)

## Euler scheme

It can be simulated in discrete time using Euler scheme by selecting a time step $∆t$ starting at $S_0, V_0$:

$$\log s_{i} = \log s_{i-1} + (r - v_{i-1}^+/2)\Delta t+ \sqrt{v_{i-1}^+ \Delta t}(  ρε^1_{i} + \sqrt{1- ρ^2} ε^2_{i})],$$

$$v_{i} =  v_{i-1} + \kappa (\bar V-v_{i-1}^+)\Delta t + \gamma \sqrt{ v_{i-1}^+ \Delta t}ε^1_{i}$$

(section 2.3 eq. 6-7 Andersen (2006) )

In [4]:
def simulate_heston_euler(state:           MarketState,
                          heston_params:   HestonParameters,
                          time:            float = 1.,
                          dt:              float = 1e-2,
                          n_simulations:   int = 10_000
                          ) -> dict:
    """Simulation engine for the Heston model using the Euler scheme.

    Args:
        state (MarketState): _description_
        heston_params (HestonParameters): _description_
        time (float, optional): _description_. Defaults to 1..
        dt (float, optional): _description_. Defaults to 1e-2.
        time_batch_size (int, optional): _description_. Defaults to 10_000.
        n_simulations (int, optional): _description_. Defaults to 10_000.

    Raises:
        Error: _description_

    Returns:
        dict: _description_
    """    
    if time<=0:
        raise Error("Time must be bigger than 0")
    
    # initialize market and model parameters
    r, s0 = state.interest_rate, state.stock_price
    
    v0, rho, kappa, vbar, gamma = heston_params.v0, heston_params.rho, heston_params.kappa, \
                                  heston_params.vbar, heston_params.gamma
    
    
    vt = np.zeros(n_simulations)
    vt[:] = v0
    log_st = np.zeros(n_simulations)
    log_st[:] = np.log(s0)
    
    N_T = int(time / dt)
    
    Z1 = np.random.normal(size=(n_simulations, N_T))
    Z2 = np.random.normal(size=(n_simulations, N_T))
    V = np.zeros([n_simulations, N_T])
    V[:, 0] = vt
    
    logS = np.zeros([n_simulations, N_T])
    logS[:, 0] = log_st

    for i in range(0,  N_T-1):
        vmax = np.maximum(V[:, i],0)
        S1 = (r - 0.5 * vmax) * (dt)
        S2 = np.sqrt(vmax*(dt)) * Z1[:, i]
        logS[:, i+1] = logS[:, i] + S1 + S2
        V1 = kappa*(vbar - vmax)*(dt)
        V2 = gamma*np.sqrt(vmax*(dt))*(rho*Z1[:, i]+np.sqrt(1-rho**2)*Z2[:, i])
        V[:, i+1] = V[:, i] + V1 + V2




    vt = V[:, N_T-1]
    log_st = logS[:, N_T-1]

        
    return {"price": np.exp(log_st), "volatility": vt}

## Andersen scheme

In [5]:
def simulate_heston_andersen_qe(state: MarketState,
                               heston_params: HestonParameters,
                               time: float = 1.,
                               dt: float = 1e-2,
                               n_simulations: int = 10_000,
                               Psi_c: float=1.5,
                               gamma_1: float=0.5) -> dict: 
    """Simulation engine for the Heston model using the Quadratic-Exponential Andersen scheme.

    Args:
        state (MarketState): _description_
        heston_params (HestonParameters): _description_
        time (float, optional): _description_. Defaults to 1..
        dt (float, optional): _description_. Defaults to 1e-2.
        n_simulations (int, optional): _description_. Defaults to 10_000.
        Psi_c (float, optional): _description_. Defaults to 1.5.
        gamma_1 (float, optional): _description_. Defaults to 0.5.

    Raises:
        Error: _description_
        Error: _description_

    Returns:
        dict: _description_
    """    
    
    if Psi_c>2 or Psi_c<1:
        raise Error('1<=Psi_c<=2 ')
    if gamma_1 >1 or gamma_1<0:
        raise Error('0<=gamma_1<=1')
        
    gamma_2 = 1.0 - gamma_1
    
    r, s0 = state.interest_rate, state.stock_price
    v0, rho, kappa, vbar, gamma = heston_params.v0, heston_params.rho, heston_params.kappa, \
                                  heston_params.vbar, heston_params.gamma
    
    
    E = np.exp(-kappa*dt)
    
    K_0 = -(rho*kappa*vbar/gamma)*dt
    K_1 = gamma_1*dt*(rho*kappa/gamma - 0.5) - rho/gamma
    K_2 = gamma_2*dt*(rho*kappa/gamma - 0.5) + rho/gamma
    K_3 = gamma_1*dt*(1.0 - rho**2)
    K_4 = gamma_2*dt*(1.0 - rho**2)
    
    N_T = int(time / dt)
    
    
    vt = np.zeros(n_simulations)
    vt[:] = v0
    log_st = np.zeros(n_simulations)
    log_st[:] = np.log(s0)
    
    mean_St = 0.
    mean_vt = 0.
    
        
    vt[:] = v0
    log_st[:] = np.log(s0)
        
    V = np.zeros([n_simulations, N_T])
    V[:, 0] = vt

    logS = np.zeros([n_simulations, N_T])
    logS[:, 0] = log_st

    Z = np.random.normal(size=(n_simulations, N_T))


    for i in range(N_T - 1):
        m=vbar+(V[:, i] - vbar)*E
        s_2 = (V[:, i]*(gamma**2)*E/kappa)*(1.0 - E) + (vbar*gamma**2)/(2.0*kappa)*((1-E)**2)

        Psi = s_2/(m**2) # np.power

        cond = np.where(Psi<=Psi_c)
        c = 2/Psi[cond]
        b = c - 1 + np.sqrt(c*(c - 1.))
        a = m[cond]/(1+b)
        b=np.sqrt(b)
        Z_V = np.random.normal(size=cond[0].shape[0])
        V[cond, i+1] = a*((b+Z_V)**2)

        cond = np.where(Psi>Psi_c)
        p = (Psi[cond] - 1)/(Psi[cond] + 1)
        beta = (1.0 - p)/m[cond]

        # ksi = np.random.binomial(1, 1.0-p, size=cond[0].shape[0])
        # eta = np.random.exponential(scale = 1/beta, size=cond[0].shape[0])
        U = np.random.uniform(0., 1. , size=cond[0].shape[0])

        V[cond,i+1]=np.where(U < p, 0., np.log((1-p)/(1-U))/beta)


        logS[:,i+1] = logS[:,i] + r*dt+K_0 + K_1*V[:,i] + K_2*V[:,i+1] \
                        + np.sqrt(K_3*V[:,i]+K_4*V[:,i+1]) * Z[:,i]

    vt = V[:, N_T-1]
    log_st = logS[:, N_T-1]
            
            
        
    #put the calculated values into the dictionary
    simulated_paths = {"price": np.exp(log_st), "volatility": vt}
    return simulated_paths

## Broadie - Kaya scheme

In [6]:
def cir_chi_sq_sample(heston_params: HestonParameters,
                      dt:            float,
                      v_i:           np.array,
                      n_simulations: int):
    """Samples chi_squared statistics for v_{i+1} conditional on 
       v_i and parameters of Hestom model. 
        
    Args:
        heston_params (HestonParameters): parameters of Heston model
        dt (float): time step 
        v_i: current volatility value
        n_simulations (int): number of simulations.
        
    Returns:
        np.array: sampled chi_squared statistics 
    """
    kappa, vbar, gamma = heston_params.kappa, heston_params.vbar, heston_params.gamma
    
    barkappa=v_i*(4*kappa*np.exp(-kappa*dt))/(gamma**2 * (1 - np.exp(-kappa*dt)))
    d=(4*kappa*vbar)/(gamma**2)
    c=((gamma**2)/(4*kappa))*(1-np.exp(-kappa*dt))
    
    return  c*np.random.noncentral_chisquare(size = n_simulations, df   = d, nonc = barkappa)


def Phi(a:             Union[float, np.ndarray], 
        V:             Union[float, np.ndarray],
        time:          Union[float, np.ndarray],
        heston_params: HestonParameters
        ) -> np.ndarray:
    
    
    v0, rho, kappa, vbar, gamma = heston_params.v0, heston_params.rho, heston_params.kappa, \
                                        heston_params.vbar, heston_params.gamma
    dt = time[1::]-time[:-1:]
    
    A=np.array(a)
    gamma_a = np.sqrt(kappa**2 - 2*gamma**2*1j*A).reshape(1,1,len(A)).T
    
    E1 = np.exp(-kappa*dt)
    E2 = np.exp(-gamma_a * dt)
        
    P1 = ((1.0-E1)*gamma_a/(kappa*(1.0-E2)))*np.exp(-0.5*(gamma_a - kappa)*dt)
    
    P2_2 = kappa * (1.0 + E1)/(1.0 - E1) - gamma_a*(1.0+E2)/(1-E2)
    P2 = np.exp( (V[:,1::]+V[:,:-1:])/(gamma_a**2) * P2_2 )
    
    P3_1 = np.sqrt(V[:,1::]*V[:,:-1:])*4*gamma_a * np.exp(-0.5*gamma_a*dt) /(gamma**2 * (1.0 - E2))
    P3_2 = np.sqrt(V[:,1::]*V[:,:-1:])*4*kappa*np.exp(-0.5*kappa*dt)/(gamma**2 * (1 - E1))
    d=(4*kappa*vbar)/(gamma**2)
    P3 = iv(0.5*d - 1, P3_1)/iv(0.5*d - 1, P3_2)
    
    return P1*P2*P3

def Pr(V:             np.ndarray, 
       time:          np.ndarray,
       X:             Union[np.ndarray, float],
       heston_params: HestonParameters,
       h:             float=1e-2, 
       eps:           float=1e-2
       ) -> np.ndarray:
    
    x=np.array(X)
    P=h*x/np.pi
    S = 0.0
    j = 1
    while(True):
        Sin=np.sin(h*j*x)/j
        Phi_hj=Phi(h*j, V, time, heston_params)
        S+=Sin.reshape(1,1,len(x)).T * Phi_hj[0]
        if np.all(Phi_hj[0]<np.pi*eps*j/2.0):
            break
        j=j+1
    
    S=S*2.0/np.pi
    return P+S

def IV(V:             np.ndarray, 
       time:          np.ndarray,
       heston_params: HestonParameters,
       h:             float=1e-2, 
       eps:           float=1e-2
       ) -> np.ndarray:
    
    U=np.random.uniform(size=(V.shape[0], V.shape[1] - 1))
    
    def f(x,i,j):
        P=Pr(V, time, x, heston_params, h, eps)
        return (P-U)[0][i,j]
    
    IVar = np.zeros((V.shape[0], V.shape[1] - 1))
    
    for i in range(IVar.shape[0]):
        for j in range(IVar.shape[1]):     
            IVar[i,j]=root_scalar(f, args=(i,j), x0=0.5, method='newton')
    return IVar

## Tests

In [15]:
heston_parameters = HestonParameters(kappa = 1.3125, gamma = 0.7125, 
                                     rho = -0.3937, vbar = 0.0641, 
                                     v0 = 0.3)

state = MarketState(stock_price = 100., interest_rate = 0.)

params = heston_parameters
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)
kwargs = {}

In [16]:
def get_payoff(maturity: float,
               strike: float,
               interest_rate: float = 0.):
    def payoff(St: np.ndarray):
        DF = np.exp(-interest_rate * maturity)
        return np.maximum(St - strike, 0)*DF

    return payoff

### At the money

In [10]:
strike = 100.
T = 1.
payoff = get_payoff(T, strike, state.interest_rate)

In [16]:
theory = model.call_price(T, strike)
print(theory)

16.173896895486777


In [23]:
np.random.seed(42)
euler    = mc_price(dt = 1e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler, params = params, T = T, **kwargs)
print(euler)

Current price: 16.1075 +/- 0.7784
Current price: 16.0910 +/- 0.6383
Current price: 16.1492 +/- 0.5548
Current price: 16.1671 +/- 0.4980
Current price: 16.0858 +/- 0.4531
Current price: 16.1576 +/- 0.4203
Current price: 16.2129 +/- 0.3947
Current price: 16.1846 +/- 0.3718
Current price: 16.1850 +/- 0.3523
Current price: 16.1878 +/- 0.3361
Current price: 16.1804 +/- 0.3214
Current price: 16.1501 +/- 0.3082
Current price: 16.1521 +/- 0.2968
Current price: 16.1584 +/- 0.2874
Current price: 16.1348 +/- 0.2774
Current price: 16.1450 +/- 0.2695
Current price: 16.1408 +/- 0.2619
Current price: 16.1517 +/- 0.2551
Current price: 16.1542 +/- 0.2487
Current price: 16.1485 +/- 0.2427
Current price: 16.1338 +/- 0.2370
Current price: 16.1507 +/- 0.2321
Current price: 16.1544 +/- 0.2275
Current price: 16.1420 +/- 0.2227
Current price: 16.1221 +/- 0.2181
Current price: 16.1150 +/- 0.2140
Current price: 16.1149 +/- 0.2102
Current price: 16.1006 +/- 0.2065
Current price: 16.1050 +/- 0.2030
Current price:

In [ ]:
np.random.seed(42)
andersen = mc_price(dt = 5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, params = params, T = T, **kwargs)
print(andersen)

Number of iterations:   552
15.96581311871001


In [13]:
for i in range(5):
    euler    = mc_price(dt = 5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler, params = params, T = T, **kwargs)
    print(euler)
print("\n\n")
for i in range(5):
    euler    = mc_price(dt = 5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, params = params, T = T, **kwargs)
    print(euler)

Number of iterations:   121
16.091897582694084
Number of iterations:   144
16.041146874432357
Number of iterations:   122
16.087502974398966
Number of iterations:   139
16.087291993347392
Number of iterations:   150
16.078932833099458



Number of iterations:   79
15.946826383478223
Number of iterations:   124
15.937841714373981
Number of iterations:   129
15.992043160725865
Number of iterations:   87
15.92852763929686
Number of iterations:   118
15.928237875583962


### Out of the money

In [ ]:
strike = 500.
payoff = get_payoff(T, strike, state.interest_rate)

In [ ]:
model.call_price(T, strike)

In [ ]:
np.random.seed(42)
euler    = mc_price(dt = 1e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler, params = params, T = T, **kwargs)
print(euler)

In [ ]:
np.random.seed(42)
andersen = mc_price(dt = 1e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen, params = params, T = T, **kwargs)
print(andersen)

### Implied volatility

In [ ]:
def d_1(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r+q*q/2)
    return denom*(log+s2)

def d_2(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r-q*q/2)
    
    return denom*(log+s2)

def calc_iv(option: CallStockOption, state: MarketState, option_price: float):
    
    T=option.expiration_time
    K=option.strike_price
    S=state.stock_price
    r=state.interest_rate    
    N=sps.norm()
    
    
    def f(q):
        d1=d_1(q,T,S,K,r)
        d2=d_2(q,T,S,K,r)
        return S*N.cdf(d1)-K*(np.exp(-r*T))*N.cdf(d2)-option_price
    
    def fprime(q):
        d1=d_1(q,T,S,K,r)
        return S*sps.norm().pdf(d1)*np.sqrt(T)
    
    sol = root_scalar(f, x0=0.5, fprime=fprime, method='newton')
    return sol.root



In [ ]:
strikes = np.arange(80, 120, 2)
strikes

In [ ]:
call_price = np.array([30.096826945107107,
 28.86569299569351,
 27.627749990421822,
 26.468755491540694,
 25.31876428557471,
 24.24782290614315,
 23.16942316862601,
 22.12935496069162,
 21.079816460198398,
 20.20119494244462,
 19.288858679686502,
 18.392107111749084,
 17.50127598061397,
 16.701957005015792,
 15.90122188123928,
 15.14196593075513,
 14.43621999411162,
 13.722786552930495,
 13.069519364876335,
 12.47747576375882])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, 
                         heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)

In [ ]:
call_price = np.zeros(20)

for j in range(20):
    call_price[j] = model.call_price(2., strikes[j])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
Z = np.random.normal(size=1000)